In [1]:
import emoji
import nltk
import numpy as np
import os
import pandas as pd
import pickle as pk
import re
import string
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision


from autocorrect import Speller
from functools import lru_cache
from matplotlib import pyplot as plt
from collections import Counter
from nltk import collocations 
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem.porter import PorterStemmer as ps
from nltk.stem import WordNetLemmatizer as lem
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, \
                                            HashingVectorizer, ENGLISH_STOP_WORDS
from sklearn.feature_selection import RFE, SelectKBest, chi2
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from stop_words import get_stop_words
from string import punctuation
from textblob import TextBlob
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
nltk.download('stopwords')
nltk.download('genesis')
nltk.download('punkt')
noise = stopwords.words('russian') + list(punctuation)
punkts = set(string.punctuation)
morph_analyz = MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xiaomi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package genesis to
[nltk_data]     C:\Users\xiaomi\AppData\Roaming\nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xiaomi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
re_url = re.compile('https?://\S+|www\.\S+')
ABC = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
ru_ABC ='абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
denial = ['not', 'cannot', 'doesn',  "wasn't", 'nor', 'couldn', 'weren', 
          "won't", "doesn't", 'needn', "isn't", 'against', "couldn't", 
          'aren', 'isn', 'hadn', 'wouldn', "mightn't", "mustn't", "can't", 
          "wouldn't", 'cant', 'didn', "shouldn't",  "don't", "didn't", 
          "hadn't", "needn't", 'shouldn', 'hasnt', "weren't", 'no', "aren't", 
          "hasn't", "haven't", 'нельзя']
stopwords_english = list(set(get_stop_words("en") + 
                             stopwords.words('english') +
                             list(ENGLISH_STOP_WORDS) +
                             ['user']).difference(denial))
stopwords_russian = list(set(get_stop_words("ru") + 
                             stopwords.words('russian') +
                             ['а','у', 'о', '☕', '♡','♥', 'б', 'аж', 'л', 'ха', 'х'
                              '❤', '❤️','❤️', '❤️', '�','✌️', 'd', '❤',
                              'dd', 'p' 'rt', 'щ', 'з', 'эй', 'ибо', '❄️']).difference(denial))

emoji_pattern = \
re.compile("[" 
           u"\U0001F600-\U0001F64F"  # emoticons
           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
           u"\U0001F680-\U0001F6FF"  # transport & map symbols
           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
           u"\U00002500-\U00002BEF"  # chinese char
           u"\U0001F191-\U0001F19A"
           u"\U0001F232-\U0001F236" u"\U0001F238-\U0001F23A" u"\U0001F250-\U0001F251"
           u"\U0001F300-\U0001F30C" u"\U0001F30D-\U0001F30E" u"\U0001F313-\U0001F315"
           u"\U0001F316-\U0001F318" u"\U0001F31D-\U0001F31E" u"\U0001F31F-\U0001F320"
           u"\U0001F32D-\U0001F32F" u"\U0001F330-\U0001F331" u"\U0001F332-\U0001F333"
           u"\U0001F334-\U0001F335" u"\U0001F337-\U0001F34A" u"\U0001F34C-\U0001F34F"
           u"\U0001F351-\U0001F37B" u"\U0001F37E-\U0001F37F" u"\U0001F380-\U0001F393"
           u"\U0001F3A0-\U0001F3C4" u"\U0001F3CF-\U0001F3D3" u"\U0001F3E0-\U0001F3E3"
           u"\U0001F3E5-\U0001F3F0" u"\U0001F3F8-\U0001F407" u"\U0001F409-\U0001F40B"
           u"\U0001F40C-\U0001F40E" u"\U0001F40F-\U0001F410" u"\U0001F411-\U0001F412"
           u"\U0001F417-\U0001F429" u"\U0001F42B-\U0001F43E" u"\U0001F442-\U0001F464"
           u"\U0001F466-\U0001F46B" u"\U0001F46C-\U0001F46D" u"\U0001F46E-\U0001F4AC"
           u"\U0001F4AE-\U0001F4B5" u"\U0001F4B6-\U0001F4B7" u"\U0001F4B8-\U0001F4EB"
           u"\U0001F4EC-\U0001F4ED" u"\U0001F4F0-\U0001F4F4" u"\U0001F4F6-\U0001F4F7"
           u"\U0001F4F9-\U0001F4FC" u"\U0001F4FF-\U0001F502" u"\U0001F504-\U0001F507"
           u"\U0001F50A-\U0001F514" u"\U0001F516-\U0001F52B" u"\U0001F52C-\U0001F52D"
           u"\U0001F52E-\U0001F53D" u"\U0001F54B-\U0001F54E" u"\U0001F550-\U0001F55B"
           u"\U0001F55C-\U0001F567" u"\U0001F595-\U0001F596" u"\U0001F5FB-\U0001F5FF"
           u"\U0001F7E0-\U0001F7EB" u"\U0001F90D-\U0001F90F" u"\U0001F910-\U0001F918"
           u"\U0001F919-\U0001F91E" u"\U0001F920-\U0001F927" u"\U0001F928-\U0001F92F"
           u"\U0001F931-\U0001F932" u"\U0001F933-\U0001F93A" u"\U0001F93C-\U0001F93E"
           u"\U0001F940-\U0001F945" u"\U0001F947-\U0001F94B" u"\U0001F94D-\U0001F94F"
           u"\U0001F950-\U0001F95E" u"\U0001F95F-\U0001F96B" u"\U0001F96C-\U0001F970"
           u"\U0001F973-\U0001F976" u"\U0001F977-\U0001F978" u"\U0001F97C-\U0001F97F"
           u"\U0001F980-\U0001F984" u"\U0001F985-\U0001F991" u"\U0001F992-\U0001F997"
           u"\U0001F998-\U0001F9A2" u"\U0001F9A3-\U0001F9A4" u"\U0001F9A5-\U0001F9AA"
           u"\U0001F9AB-\U0001F9AD" u"\U0001F9AE-\U0001F9AF" u"\U0001F9B0-\U0001F9B9"
           u"\U0001F9BA-\U0001F9BF" u"\U0001F9C1-\U0001F9C2" u"\U0001F9C3-\U0001F9CA"
           u"\U0001F9CD-\U0001F9CF" u"\U0001F9D0-\U0001F9E6" u"\U0001F9E7-\U0001F9FF"
           u"\U0001FA70-\U0001FA73" u"\U0001FA78-\U0001FA7A" u"\U0001FA7B-\U0001FA7C"
           u"\U0001FA80-\U0001FA82" u"\U0001FA83-\U0001FA86" u"\U0001FA90-\U0001FA95"
           u"\U0001FA96-\U0001FAA8" u"\U0001FAA9-\U0001FAAC" u"\U0001FAB0-\U0001FAB6"
           u"\U0001FAB7-\U0001FABA" u"\U0001FAC0-\U0001FAC2" u"\U0001FAC3-\U0001FAC5"
           u"\U0001FAD0-\U0001FAD6" u"\U0001FAD7-\U0001FAD9" u"\U0001FAE0-\U0001FAE7"
           u"\U0001FAF0-\U0001FAF6" u"\U00002702-\U000027B0" u"\U00002702-\U000027B0" 
           u"\U000024C2-\U0001F251" u"\U0001f926-\U0001f937" u"\U00010000-\U0010ffff"
           u"\u231A-\u231B" u"\u200d" u"\u23E9-\u23EC" u"\u25FD-\u25FE" u"\u2600-\u2B55" 
           u"\ufe0f"  # dingbats
           u"\u3030" "]+", flags=re.UNICODE)

apostrophe_dict = {
            "ain`t": "am not / are not",
            "aren`t": "are not / am not",
            "can`t": "cannot",
            "can`t`ve": "cannot have",
            "`cause": "because",
            "could`ve": "could have",
            "couldn`t": "could not",
            "couldn`t`ve": "could not have",
            "didn`t": "did not",
            "doesn`t": "does not",
            "don`t": "do not",
            "hadn`t": "had not",
            "hadn`t've": "had not have",
            "hasn`t": "has not",
            "haven`t": "have not",
            "he`d": "he had / he would",
            "he`d`ve": "he would have",
            "he`ll": "he shall / he will",
            "he`ll`ve": "he shall have / he will have",
            "he`s": "he has / he is",
            "how`d": "how did",
            "how`d'y": "how do you",
            "how`ll": "how will",
            "how`s": "how has / how is",
            "i`d": "I had / I would",
            "i`d`ve": "I would have",
            "i`ll": "I shall / I will",
            "i`ll`ve": "I shall have / I will have",
            "i`m": "I am",
            "i`ve": "I have",
            "isn`t": "is not",
            "it`d": "it had / it would",
            "it`d`ve": "it would have",
            "it`ll": "it shall / it will",
            "it`ll`ve": "it shall have / it will have",
            "it`s": "it has / it is",
            "let`s": "let us",
            "ma`am": "madam",
            "mayn`t": "may not",
            "might`ve": "might have",
            "mightn`t": "might not",
            "mightn`t`ve": "might not have",
            "must`ve": "must have",
            "mustn`t": "must not",
            "mustn`t`ve": "must not have",
            "needn`t": "need not",
            "needn`t`ve": "need not have",
            "o`clock": "of the clock",
            "oughtn`t": "ought not",
            "oughtn`t`ve": "ought not have",
            "shan`t": "shall not",
            "sha`n`t": "shall not",
            "shan`t`ve": "shall not have",
            "she`d": "she had / she would",
            "she`d`ve": "she would have",
            "she`ll": "she shall / she will",
            "she`ll`ve": "she shall have / she will have",
            "she`s": "she has / she is",
            "should`ve": "should have",
            "shouldn`t": "should not",
            "shouldn`t`ve": "should not have",
            "so`ve": "so have",
            "so`s": "so as / so is",
            "that`d": "that would / that had",
            "that`d've": "that would have",
            "that`s": "that has / that is",
            "there`d": "there had / there would",
            "there`d`ve": "there would have",
            "there`s": "there has / there is",
            "they`d": "they had / they would",
            "they`d`ve": "they would have",
            "they`ll": "they shall / they will",
            "they`ll`ve": "they shall have / they will have",
            "they`re": "they are",
            "they`ve": "they have",
            "to`ve": "to have",
            "wasn`t": "was not",
            "we`d": "we had / we would",
            "we`d`ve": "we would have",
            "we`ll": "we will",
            "we`ll`ve": "we will have",
            "we`re": "we are",
            "we`ve": "we have",
            "weren`t": "were not",
            "what`ll": "what shall / what will",
            "what`ll`ve": "what shall have / what will have",
            "what`re": "what are",
            "what`s": "what has / what is",
            "what`ve": "what have",
            "when`s": "when has / when is",
            "when`ve": "when have",
            "where`d": "where did",
            "where`s": "where has / where is",
            "where`ve": "where have",
            "who`ll": "who shall / who will",
            "who`ll`ve": "who shall have / who will have",
            "who`s": "who has / who is",
            "who`ve": "who have",
            "why`s": "why has / why is",
            "why`ve": "why have",
            "will`ve": "will have",
            "won`t": "will not",
            "won`t`ve": "will not have",
            "would`ve": "would have",
            "wouldn`t": "would not",
            "wouldn`t`ve": "would not have",
            "y`all": "you all",
            "y`all`d": "you all would",
            "y`all`d`ve": "you all would have",
            "y`all`re": "you all are",
            "y`all`ve": "you all have",
            "you`d": "you had / you would",
            "you`d`ve": "you would have",
            "you`ll": "you shall / you will",
            "you`ll`ve": "you shall have / you will have",
            "you`re": "you are",
            "you`ve": "you have"
        }

@lru_cache(maxsize=2048)
def text_preprocess_light(text):
    text = str(text)
    text = re_url.sub('', text)
    nicks = re.findall(r'@[\w]*', text)
    for nick in nicks:
        text = re.sub(nick, ' ', text)
    re_html = re.compile('<.*?>')
    text = re_html.sub(r'', text)
    text = emoji_pattern.sub(r'',text)
    for word in text.split():
        if word.lower() in apostrophe_dict:
            text = re.sub(word, apostrophe_dict[word.lower()], text)
    text = ' '.join([word for word in text.split() if len(word) > 1])
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'[^а-яА-Я]', ' ', text)
    text = re.sub(r'\s\s+', ' ', text)
    text = re.sub('не\s', 'не', text)
    text = re.sub('нет\s', 'нет', text)
    text = ' '.join([word for word in text.split() if len(word) > 2])
    text = text.lower()
#     text = TextBlob(text).correct().string
    text = [morph_analyz.parse(word)[0].normal_form for word in text.split() if word not in stopwords_russian] 
                                                                           # if word not in stopwords_english
    return " ".join(text)